<a href="https://colab.research.google.com/github/Muhammad-Danish-Abbas/Agentic_AI/blob/main/Project_2_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata

# Get the API key and remove any leading/trailing whitespace and newline characters
api_key = userdata.get('GOOGLE_API_KEY').strip()

genai.configure(api_key=api_key)

from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content="What is the meaning of life?",
    task_type="retrieval_document",
    title="Embedding of single string",
)

# # 1 input > 1 vector output
# print(str(result["embedding"])[:50], "... TRIMMED]")

result['embedding']

[-0.02854543,
 0.044588115,
 -0.034197364,
 -0.0042663575,
 -0.04079577,
 0.012999958,
 0.018053582,
 0.06015144,
 -0.0028713925,
 0.009951648,
 0.024832657,
 -0.01683923,
 0.09940116,
 -0.031990346,
 0.018328529,
 -0.109134205,
 0.001190296,
 0.0014311911,
 -0.083155245,
 -0.010203233,
 0.019211812,
 0.0010217889,
 0.053874534,
 -0.0150861535,
 -0.003189089,
 0.019626662,
 -0.0074312133,
 -0.036586244,
 -0.008509182,
 -0.017352631,
 0.058202818,
 0.05446324,
 0.01571296,
 -0.021822602,
 0.048009068,
 0.022641798,
 -0.0069730366,
 0.054272633,
 0.025922865,
 -0.027334303,
 -0.07256842,
 0.028509492,
 -0.03564165,
 0.060492564,
 -0.022731686,
 -0.030770157,
 -0.006176277,
 -0.021891864,
 -0.019659325,
 0.0643669,
 0.03154234,
 0.017379418,
 -0.03679774,
 0.016511764,
 -0.02536976,
 -0.022270117,
 -0.012396498,
 -0.032805424,
 0.054154944,
 -0.04823156,
 -0.021759441,
 -0.03370158,
 -0.025460402,
 -0.017531719,
 -0.052902102,
 0.04005264,
 -0.022417234,
 0.023286799,
 -0.081740536,
 0.05

In [ ]:
len(result['embedding'])

768

In [ ]:
from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content=[
        "What is the meaning of life?",
        "How much wood would a woodchuck chuck?",
        "How does the brain work?",
    ],
    task_type="retrieval_document",
    title="Embedding of list of strings",
)

# A list of inputs > A list of vectors output
for v in result["embedding"]:
    print(str(v)[:50], "... TRIMMED ...", len(v))

[-0.036453027, 0.033254996, -0.03970925, -0.002628 ... TRIMMED ... 768
[-0.01591948, 0.032582663, -0.081024624, -0.011298 ... TRIMMED ... 768
[0.00037063024, 0.03763057, -0.122695684, -0.00951 ... TRIMMED ... 768


#Chroma_DB and Langchain

In [ ]:
!pip install -Uq langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.0 MB/s eta 0:00:00


In [ ]:
import getpass
import os

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os # This line imports the 'os' module.


os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-004")


In [ ]:
embeddings_embed.query("wHAT IS THIS")

NameError: name 'embeddings_embed' is not defined

In [ ]:
from google.colab import userdata
import os

# Get the API key and remove any leading/trailing whitespace and newline characters
api_key = userdata.get('GOOGLE_API_KEY')
if api_key is None or api_key.strip() == '':
    # If api_key is None, then the user has to store their API key in userdata
    print("Please store your Google API key in Colab userdata.")
    print("You can do this by running the following code:")
    print("from google.colab import userdata")
    print("userdata.set('GOOGLE_API_KEY', 'YOUR_API_KEY')")
else:
    # Otherwise the api_key is set
    os.environ['GOOGLE_API_KEY'] = api_key.strip()
    print("API key set in environment variables.")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("What is my name")
print(vector)

In [ ]:
embeddings.embed_query("What is my name")

[0.016247833147644997,
 -0.035366397351026535,
 -0.005839777644723654,
 -0.036411359906196594,
 0.04308868572115898,
 0.03013709932565689,
 -0.0007378686568699777,
 0.013127364218235016,
 -0.004319014027714729,
 0.004911960568279028,
 -0.0026811622083187103,
 0.004515391308814287,
 -0.02232649363577366,
 -0.055035706609487534,
 0.018942922353744507,
 0.008696216158568859,
 0.01006714440882206,
 0.021661309525370598,
 0.011926365084946156,
 -0.021640487015247345,
 -0.03030930645763874,
 0.03685571253299713,
 -0.033659547567367554,
 -0.04743378236889839,
 0.016980625689029694,
 -0.0352543406188488,
 0.024664858356118202,
 -0.04842172563076019,
 -0.04000058025121689,
 0.06503179669380188,
 -0.046474114060401917,
 0.012603935785591602,
 -0.07052986323833466,
 0.00861891359090805,
 0.024031607434153557,
 -0.07729999721050262,
 0.03365229815244675,
 0.0219412911683321,
 0.03792271763086319,
 0.015776894986629486,
 -0.006878132000565529,
 -0.026465823873877525,
 -0.06492599099874496,
 -0.0234

In [ ]:
!pip install -U langchain-community # Install the langchain-community package
from langchain.vectorstores import Chroma # Now you can import Chroma from langchain.vectorstores
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from uuid import uuid4

# ... (Rest of the code remains the same)

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)
document_11 = Document(
    page_content="Life is the collection of experinces",
    metadata={"source": "website"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
    document_11,
]

# Initialize the embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create the vectorstore
vectorstore = Chroma.from_documents(documents, embeddings)

# Now you can list the directory of the vectorstore
list(dir(vectorstore))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00


['_Chroma__query_collection',
 '_LANGCHAIN_DEFAULT_COLLECTION_NAME',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_asimilarity_search_with_relevance_scores',
 '_client',
 '_client_settings',
 '_collection',
 '_cosine_relevance_score_fn',
 '_embedding_function',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_persist_directory',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_images',
 'add_texts',
 'adelete',
 'afrom_documents'

In [ ]:
len(documents)

11

In [ ]:
from uuid import uuid4
uuid4()

UUID('30defffc-d587-44cb-bcce-1ca0d833b7f3')

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

# Use 'vectorstore' instead of 'vector_store'
vectorstore.add_documents(documents=documents, ids=uuids)

['83966b15-8d4a-4532-ad3c-0a581c1c610e',
 '679aa0c0-6059-4c2d-9d6c-efadcdddaa67',
 '58c10680-60b8-4e81-9b97-53bdeda591eb',
 '864f44b3-e4d3-40c7-a4b3-8f8168f00e66',
 'ae29043d-a00c-42bc-971f-e683dcbae5e8',
 '6d8213eb-a5b5-41f9-b3c4-c3ff05293316',
 'eadd34dc-706c-442b-8628-0faf20b5966f',
 '682b82a7-f76a-4ccb-a67e-e1953676b954',
 '044c8682-2d9a-4a25-a469-54e9ca49bc58',
 'fb129767-41ba-4af6-90ed-7cfd700d65a3',
 '065e10b5-8269-46d6-b9a8-5b3e5223f1f3']

In [ ]:
vectorstore

In [ ]:
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'website'}, page_content='Life is the collection of experinces'),
 Document(metadata={'source': 'website'}, page_content='Life is the collection of experinces')]

In [ ]:
# Note that providers implement different scores; Chroma here
# returns a distance metric that should vary inversely with
# similarity.

vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
  0.8349317312240601),
 (Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
  0.8349317312240601),
 (Document(metadata={'source': 'website'}, page_content='Life is the collection of experinces'),
  0.8676630258560181),
 (Document(metadata={'source': 'website'}, page_content='Life is the collection of experinces'),
  0.8676630258560181)]

In [ ]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001").embed_query("cat")

vectorstore.similarity_search_by_vector(embedding)

[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'website'}, page_content='Life is the collection of experinces'),
 Document(metadata={'source': 'website'}, page_content='Life is the collection of experinces')]

#Retrievers

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["shark"])

[[Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')]]

#Data Retrievers


In [ ]:
results = vectorstore.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy and tell about Life" ,
   k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [ ]:
results = vectorstore.similarity_search_with_score(
    "Will it be hot tomorrow?", k=2, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.664568] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.664568] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [ ]:
def answer_to_user(query: str):
    # Perform similarity search to get the most relevant result
    results = vectorstore.similarity_search(query, k=1)

    if results:  # Ensure results exist
        # Extract content from the search result
        context = results[0].page_content  # Use .page_content to access the content

        # Use a language model instead of the image embedding model
        # Replace with your preferred language model and its usage
        # Example: Assuming you have a language model called 'language_model'
        final_answer = f"Context: {context}\nQuestion: {query}\nAnswer: Placeholder for language model answer"
        # final_answer = language_model(f"Context: {context}\nQuestion: {query}")
        return final_answer
    else:
        return "Sorry, I couldn't find a relevant answer."

In [ ]:
pip install langchain_openai faiss-cpu


In [ ]:
user_query = "What is the capital of China?"
answer = answer_to_user(user_query)
print(answer)


Context: I have a bad feeling I am going to get deleted :(
Question: What is the capital of China?
Answer: Placeholder for language model answer


#Face detection with embedding

In [1]:
!pip install -Uq facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.2.2 which is incompatible.


In [2]:
!pip install -Uq pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 24.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires Pillow<10.3.0,>=10.2.0, but you have pillow 11.1.0 which is incompatible.


In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

import torchvision.transforms as transforms
from PIL import Image

In [4]:
from facenet_pytorch import MTCNN, InceptionResnetV1

model = InceptionResnetV1(pretrained='vggface2').eval()
model

  0%|          | 0.00/107M [00:00<?, ?B/s]

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [5]:
# Preprocessing function to transform the image into a tensor
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

# Function to create image embeddings
def create_image_embedding(image_path):
    try:
        input_tensor = preprocess_image(image_path)
        with torch.no_grad():
            embeddings = model(input_tensor)# ebedding important line
        return embeddings.squeeze().numpy()
    except Exception as e:
        print("Error:", e)
        return None

In [6]:
!mkdir images

In [7]:
# prompt: create python function where we provide image url and imag_name then it save in images folder

import requests
import os

def save_image_from_url(image_url, image_name):
  """
  Downloads an image from a URL and saves it to the 'images' folder.

  Args:
    image_url: The URL of the image to download.
    image_name: The name of the file to save the image as.
  """
  try:
    if not os.path.exists("images"):
      os.makedirs("images")

    image_path = os.path.join("images", image_name)

    response = requests.get(image_url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(image_path, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"Image saved to: {image_path}")
  except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
  except Exception as e:
    print(f"An error occurred: {e}")


In [8]:
save_image_from_url("https://media.licdn.com/dms/image/v2/D4E03AQEEn9DuNlQwvw/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1664654245747?e=2147483647&v=beta&t=NGB0a9aqsgdyxpbuO3rqws95ogJnL_6aRtBDS7IWPfw","s1.jpg")
save_image_from_url("https://avatars.githubusercontent.com/u/10209765?v=4", "q1.jpg")

save_image_from_url("https://media.licdn.com/dms/image/v2/D4D22AQFmuEiR8ttUmw/feedshare-shrink_800/feedshare-shrink_800/0/1711203894556?e=2147483647&v=beta&t=GEZGp_cdogNJCJIGidoEw_DjW2FXZcG4nUUlaNF1Zlc","z1.jpg")
save_image_from_url("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBBiqefc7Le97Rn0udVVBkur7RlU53FcQh1A&s",'z2.jpg')
save_image_from_url("https://scontent.fkhi4-4.fna.fbcdn.net/v/t39.30808-6/468785380_10160566910882765_300507882801991935_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=6ee11a&_nc_eui2=AeEk77SJKagGymTo3ibNnnx9YsjCm8DJ0lRiyMKbwMnSVMJqs7YWsJDuzKzXyLHLoFk&_nc_ohc=QJMm9K-AE4QQ7kNvgFE0N2o&_nc_oc=Adi1r8eogMcuDIMMLJvliCOnaaXQ2KnUbbJvY94aAnfInkDB-fyB_1ZXBpDQnWTkZnY&_nc_zt=23&_nc_ht=scontent.fkhi4-4.fna&_nc_gid=AcSynbwc6ukNTxWnUzjhtEe&oh=00_AYCvnF-vj63T-X69PZgBk6JvVjepzybVukgHPSM_6BXkGQ&oe=678003A4",'s2.jpg')
save_image_from_url("https://i.ytimg.com/vi/7QD3GKvSyMk/hqdefault.jpg?sqp=-oaymwEmCOADEOgC8quKqQMa8AEB-AHOBYAC0AWKAgwIABABGGUgXChPMA8=&rs=AOn4CLB2EaZsLrClGHqUMUhApQ_sxAcF7Q","q2.jpg")

Image saved to: images/s1.jpg
Image saved to: images/q1.jpg
Image saved to: images/z1.jpg
Image saved to: images/z2.jpg
Error downloading image: 403 Client Error: Forbidden for url: https://scontent.fkhi4-4.fna.fbcdn.net/v/t39.30808-6/468785380_10160566910882765_300507882801991935_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=6ee11a&_nc_eui2=AeEk77SJKagGymTo3ibNnnx9YsjCm8DJ0lRiyMKbwMnSVMJqs7YWsJDuzKzXyLHLoFk&_nc_ohc=QJMm9K-AE4QQ7kNvgFE0N2o&_nc_oc=Adi1r8eogMcuDIMMLJvliCOnaaXQ2KnUbbJvY94aAnfInkDB-fyB_1ZXBpDQnWTkZnY&_nc_zt=23&_nc_ht=scontent.fkhi4-4.fna&_nc_gid=AcSynbwc6ukNTxWnUzjhtEe&oh=00_AYCvnF-vj63T-X69PZgBk6JvVjepzybVukgHPSM_6BXkGQ&oe=678003A4
Image saved to: images/q2.jpg


In [9]:
# Example usage
image_path = "./images/q2.jpg"
q2 = create_image_embedding(image_path)

# 'embedding' now contains a dense vector representation of the image
print("Image Embedding Shape:", q2.shape)
print("Image Embedding:", q2)

Image Embedding Shape: (512,)
Image Embedding: [ 0.00058278 -0.01463297 -0.10330155  0.03984535  0.08525636  0.08029363
 -0.01011138  0.04867227 -0.00259888 -0.0077198  -0.0331905   0.06225294
  0.01708761 -0.00982595 -0.01638006  0.00054205  0.04426373  0.00519726
  0.04448376 -0.08979201 -0.06033407 -0.00162232  0.09186839  0.03262272
  0.06873461  0.02532319  0.03512866 -0.05809444  0.00151727  0.05172818
 -0.04107905 -0.03755797 -0.04134395 -0.00761167 -0.00674731  0.04443471
  0.00570058 -0.02845279 -0.12002273  0.05528227  0.00132485  0.00437373
 -0.02495593  0.01842853 -0.00357915  0.01889837  0.02980521  0.10357354
 -0.11238679 -0.02633037  0.02794239 -0.0152881   0.04015452 -0.0039458
 -0.09202526  0.0715652  -0.03382103  0.07384298  0.01921727 -0.04263638
  0.02758526  0.03841295 -0.04435986 -0.05347932 -0.04511738  0.03700717
 -0.00194125 -0.00450149  0.03157888  0.02796174  0.02416223  0.01202647
 -0.00905218  0.00880154  0.0189873  -0.01363976 -0.0499587  -0.01557556
  0.0

In [10]:
image_path = "./images/s2.jpg"
cat2 = create_image_embedding(image_path)

# 'embedding' now contains a dense vector representation of the image
print("Image Embedding Shape:", cat2.shape)
print("Image Embedding:", cat2)


Error: [Errno 2] No such file or directory: './images/s2.jpg'


AttributeError: 'NoneType' object has no attribute 'shape'

In [11]:
q1 = create_image_embedding("./images/q1.jpg")
q2 = create_image_embedding("./images/q2.jpg")
s1 = create_image_embedding("./images/s1.jpg")
s2 = create_image_embedding("./images/s2.jpg")
z1 = create_image_embedding("./images/z1.jpg")
z2 = create_image_embedding("./images/z2.jpg")

Error: [Errno 2] No such file or directory: './images/s2.jpg'


In [12]:
!pip install -U milvus-lite

!pip install -U pymilvus


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.69.0
    Uninstalling grpcio-1.69.0:
      Successfully uninstalled grpcio-1.69.0


In [13]:
from pymilvus import MilvusClient
client = MilvusClient("./milvus_demo.db")


In [14]:
from pymilvus import MilvusClient
import numpy as np

client = MilvusClient("./milvus_demo.db")
client.create_collection(
    collection_name="images",
    dimension=512  # The vectors we will use in this demo has 384 dimensions
)

In [37]:
import torch
import torch.nn as nn
import torchvision.models as models

import torchvision.transforms as transforms
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1

model = InceptionResnetV1(pretrained='vggface2').eval()

# Preprocessing function to transform the image into a tensor
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

# Function to create image embeddings
def create_image_embedding(image_path):
    try:
        input_tensor = preprocess_image(image_path)
        with torch.no_grad():
            embeddings = model(input_tensor)# ebedding important line
        return embeddings.squeeze().numpy()
    except Exception as e:
        print(f"Error processing image {image_path}: {e}") # Print the image path causing the error
        return None

# Define q1, q2, s1, s2, z1, z2 before you use them in the 'data' list
q1 = create_image_embedding("./images/q1.jpg") # Defining q1
q2 = create_image_embedding("./images/q2.jpg")
s1 = create_image_embedding("./images/s1.jpg")
s2 = create_image_embedding("./images/s2.jpg")
z1 = create_image_embedding("./images/z1.jpg")
z2 = create_image_embedding("./images/z2.jpg")

# Filter out None values from the data list
data = [
    {"id": 1, "person_name": "Qasim", "vector": q1},
    {"id": 2, "person_name": "Qasim", "vector": q2},
    {"id": 3, "person_name": "Shahzad", "vector": s1},
    {"id": 4, "person_name": "Shahzad", "vector": s2},
    {"id": 5, "person_name": "Zia Khan", "vector": z1},
    {"id": 6, "person_name": "Danish", "vector": q3},
    {"id": 7, "person_name": "Zia Khan", "vector": z2}
]
data = [d for d in data if d['vector'] is not None] # Remove entries with None vectors

Error processing image ./images/s2.jpg: [Errno 2] No such file or directory: './images/s2.jpg'


In [18]:
res = client.insert(
    collection_name="images",
    data=data
)

In [38]:
res = client.search(
    collection_name="images",
    data=[s1],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)


data: ["[{'id': 1, 'distance': 0.43397319316864014, 'entity': {'person_name': 'Qasim', 'id': 1}}]"]


In [35]:
q3 = create_image_embedding('./images/q3.jpg')



In [33]:
res = client.search(
    collection_name="images",
    data=[q2],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)

data: ["[{'id': 2, 'distance': 1.0000001192092896, 'entity': {'person_name': 'Qasim', 'id': 2}}]"]
